In [68]:
import pandas as pd
import re

In [ ]:
feedbacks = pd.read_csv('./old/objetivas.csv')

feedbacks.drop(columns=['curso', 'disciplina'], inplace=True)

feedbacks = feedbacks[feedbacks['Média do Professor'].notna() & feedbacks['Avaliação e feedback'].notna() & feedbacks['Aspectos operacionais'].notna() & feedbacks['Aprendizado Centrado no Aluno'].notna() & feedbacks['Ensino'].notna()]

feedbacks['responses_relative'] = feedbacks['Quantidade de formulários Respondidos'] / feedbacks['Quantidade de Alunos']

feedbacks['professor'] = feedbacks['professor'].apply(lambda x: x.upper())

feedbacks

In [ ]:

discursivas_old = pd.read_csv('./old/discursivas.csv')

grouped_continue_doing = discursivas_old.groupby(['Grupo de Turma', 'Professor', 'Avaliação'])['Disc1'].apply(lambda x: x.dropna().tolist()).reset_index()
grouped_stop_doing = discursivas_old.groupby(['Grupo de Turma', 'Professor', 'Avaliação'])['Disc2'].apply(lambda x: x.dropna().tolist()).reset_index()
grouped_start_doing = discursivas_old.groupby(['Grupo de Turma', 'Professor', 'Avaliação'])['Disc3'].apply(lambda x: x.dropna().tolist()).reset_index()

# Faz o merge com o dataframe discursivas_old
discursivas_old = pd.merge(discursivas_old, grouped_continue_doing, on=['Grupo de Turma', 'Professor', 'Avaliação'], suffixes=('', '_list'))
discursivas_old = pd.merge(discursivas_old, grouped_stop_doing, on=['Grupo de Turma', 'Professor', 'Avaliação'], suffixes=('', '_list'))
discursivas_old = pd.merge(discursivas_old, grouped_start_doing, on=['Grupo de Turma', 'Professor', 'Avaliação'], suffixes=('', '_list'))

# Remove a coluna original continue_doing
discursivas_old.drop(columns=['Disc1'], inplace=True)
discursivas_old.drop(columns=['Disc2'], inplace=True)
discursivas_old.drop(columns=['Disc3'], inplace=True)
discursivas_old.drop(columns='Família', inplace=True)

# Renomeia a coluna agrupada
discursivas_old.rename(columns={'Disc1_list': 'continue_doing'}, inplace=True)
discursivas_old.rename(columns={'Disc2_list': 'stop_doing'}, inplace=True)
discursivas_old.rename(columns={'Disc3_list': 'start_doing'}, inplace=True)
discursivas_old.rename(columns={'Grupo de Turma': 'codigo_turma'}, inplace=True)
discursivas_old.rename(columns={'Professor': 'professor'}, inplace=True)

discursivas_old['continue_doing'] = discursivas_old['continue_doing'].apply(tuple)
discursivas_old['stop_doing'] = discursivas_old['stop_doing'].apply(tuple)
discursivas_old['start_doing'] = discursivas_old['start_doing'].apply(tuple)

# Separa o ano do período
discursivas_old['Ano-Per'] = discursivas_old['Ano-Per'].apply(lambda x: str(x))
discursivas_old['ano'] = discursivas_old['Ano-Per'].apply(lambda x: x.split('.')[0])
discursivas_old['periodo'] = discursivas_old['Ano-Per'].apply(lambda x: x.split('.')[1][-1])

# Remove a coluna Ano-Per
discursivas_old.drop(columns=['Ano-Per'], inplace=True)

discursivas_old['ano'] = discursivas_old['ano'].astype(int)
discursivas_old['periodo'] = discursivas_old['periodo'].astype(int)

discursivas_old.drop_duplicates(inplace=True)
discursivas_old.sort_values('codigo_turma', inplace=True)
discursivas_old.reset_index(inplace=True, drop=True)

discursivas_old['Formulário'] = discursivas_old['Formulário'].replace({
    681: 'P',
    118: 'F'
})
discursivas_old.rename(columns={'Formulário': 'final/parcial'}, inplace=True)

discursivas_old = discursivas_old[['ano', 'periodo', 'final/parcial', 'codigo_turma', 'professor', 'continue_doing', 'stop_doing', 'start_doing']]

discursivas_old.sort_values(['professor', 'ano', 'periodo'], inplace=True)

discursivas_old

In [ ]:
merged_surveys = pd.merge(feedbacks, discursivas_old, on=['codigo_turma', 'ano', 'periodo', 'professor', 'final/parcial'])

merged_surveys

In [ ]:
# Carrega os dados dos planos de aula
lesson_plans = pd.read_csv('../lesson-plans/planos_pgls_all.csv')

# Mantém apenas as colunas necessárias
lesson_plans = lesson_plans.drop(columns=['id'])

lesson_plans['periodo'] = lesson_plans['periodo'].apply(lambda x: str(x)[1]).astype(int)

# Função para extrair a turma
def extrair_turma_e_divisao(codigo):
    # Expressão regular para extrair a turma (letras e números iniciais)
    match_turma = re.match(r'^[A-Za-z]+\d+', codigo)
    turma = match_turma.group(0) if match_turma else codigo

    # Expressão regular para extrair a divisão (últimos caracteres após "_", se existir)
    match_divisao = re.search(r'_(\w+)$', codigo)
    divisao = match_divisao.group(1) if match_divisao else None

    # Retorna a turma com a divisão anexada, se existir
    return f"{turma}_{divisao}" if divisao else turma

turma = lesson_plans['codigo_turma'].apply(extrair_turma_e_divisao)

lesson_plans['turma'] = turma

lesson_plans['professores'] = lesson_plans['professores'].str.split(',')
lesson_plans = lesson_plans.explode('professores').reset_index(drop=True)
lesson_plans['professores'] = lesson_plans['professores'].str.strip()
lesson_plans.rename(columns={'professores': 'professor'}, inplace=True)

lesson_plans['professor'] = lesson_plans['professor'].apply(lambda x: x.upper())

lesson_plans = lesson_plans.drop(columns=['email_professores'])

lesson_plans

In [ ]:
joined = pd.merge(lesson_plans, merged_surveys, on=['ano', 'periodo', 'codigo_turma', 'professor'])

joined.sort_values(['ano', 'periodo'], ascending=False, inplace=True)

joined = joined[
    [
        'ano', 
        'periodo', 
        'curso',
        'disciplina',
        'turma',
        'codigo_turma',
        'professor',
        'final/parcial',
        'link',
        'autoavaliacao_aluno',
        'Ensino',
        'Aprendizado Centrado no Aluno',
        'Aspectos operacionais',
        'Avaliação e feedback',
        'Avaliação da monitoria',
        'Média',
        'Média do Professor',
        'Quantidade de Alunos',
        'Quantidade de formulários Respondidos',
        'responses_relative',
        'Recomenda o professor/curso',
        'Não recomenda o professor/curso',
        'continue_doing',
        'stop_doing',
        'start_doing'
    ]
]

joined.sort_values(['ano', 'periodo', 'professor', 'turma'], ascending=False, inplace=True)

joined.reset_index(drop=True, inplace=True)
 
joined.to_csv('surveys_old.csv', index=False)

joined